# 5. Helpful SLURM Commands
David John Gagne

SLURM is currently the scheduler on the Casper cluster, which means it is used to manage the queueing and scheduling of jobs. You are likely very familiar with sbatch and squeue at this point. SLURM also has a whole suite of other commands that give you an incredibly detailed view into the usage of the cluster by yourself and everyone else. This blog will provide some insights to help you better manage your own jobs and keep track of how busy Casper is.

## Track your job memory and CPU usage with sacct
`sacct` queries the SLURM scheduler database to find out how well you or any other user has utilized their requested resources on a job by job basis. The default output of sacct is not very useful, but with a few alterations to the command, you can get a wealth of information.

I recommend running sacct in the following format (Note that ! allows you to run a command line program within a notebook. Do not copy the ! if you want to use the command in the terminal window):

In [2]:
! sacct --units=G --format="User,JobName,JobID,AllocNodes,ReqCPUs,Elapsed,CPUTime,TotalCPU,ReqMem,MaxRSS,ExitCode,State" -S 2020-12-01 -E 2020-12-31 -u dgagne

/bin/sh: sacct: command not found


The command breaks down into these parts:
- `--units=G`: Print all memory-related outputs in Gigabytes. You can also use M or K for Megabytes and Kilobytes
- `--format="..."`: The list of columns to output. The full list can be found [here](https://slurm.schedmd.com/sacct.html). 
- `-S 2020-12-01`: The start date for the query. Can be adjusted so only recent jobs are visible.
- `-E 2020-12-31`: The end date for the query. 
- `-u dgagne`: The username. Can be a comma separated list of users, like `-u dgagne,cbecker,schreck,ggantos`

What does the output mean? The most relevant comparisons relate to CPU and memory usage. 
- Elapsed: total time the job runs in Day-Hour:Minute:Second format.
- CPUTime: total time the CPUs are allocated, which should be close to Elapsed * ReqCPUs. 
- TotalCPU: The total amount of time the CPUs are in use by the user or the system. If this is far less than CPUTime, then you are requesting too many CPUs for your job. Note that TotalCPU does not account for child processes, so if you are running multiprocessing or dask, this number may be deceptively low. 

For memory usage
- ReqMem: The total amount of memory the job requested.
- MaxRSS: The maximum amount of memory the job used. If MaxRSS is far less than ReqMem, then decrease future memory requests. If it is the same or close to the same as ReqMem and your job is taking a longer than expected time to run, the program may be swapping memory to disk. You should ask for more memory in that case. 

## Track current cluster usage with sinfo
`sinfo` prints out information about the current usage of every node in the cluster. It is helpful to see which nodes have what resources, and you can see how busy each node is. This may be especially useful when you are about to launch a multi-GPU or large memory job and want to make sure the memory and GPUs are available. The default `sinfo` call provides a very high level summary. Just like `sacct`, I recommend running the following command:


In [3]:
! sinfo --Format="NodeHost:15,Features:50,CPUs:5,CPUsLoad:10,Gres:30,GresUsed:50,AllocMem:.15,FreeMem:.15  ,StateLong:15,Available:6"

/bin/sh: sinfo: command not found


The columns perform the following uses:
- NodeHost: Prints the name of each node.
- Features: Lists the CPU type (skylake or cascadelake), and the number and type of GPU if any
- CPUs: Number of CPUs available, which is number of sockets * number of cores * threads per core (only for multithreading)
- CPUsLoad: How many CPUs are currently being used
- Gres: Number and type of GPUs
- GresUsed: How many GPUs are currently allocated on the node
- AllocMem: How much memory is allocated in MB
- FreeMem: How much memory is free in MB
- StateLong: Node usage, which can be idle, mixed, allocated, reserved, or drained
- Available: up or down